In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2


In [ ]:
def show(imgs: list) -> None:
    fig, axes = plt.subplots(nrows=1, ncols=5, figsize=(20, 5))

    for img, ax in zip(imgs, axes.ravel()):
        ax.imshow(img, cmap="gray", interpolation='antialiased', aspect='equal')
        ax.axis("off")

    fig.tight_layout()

# Loading dataset
It returns a list of grayscale images that can be iterated. Grayscale will increase accuracy removing neglecting colours.

In [ ]:
def get_data(path: str) -> list:
    imgs = os.listdir(path)
    return [cv2.imread(f"{path}/{img}", 0) for img in imgs]

In [ ]:
small_data = get_data("data/local")[1:6]
show(small_data)

# Blur
Blur will remove noise and artefacts from the camera

In [ ]:
def blur_image(img: np.ndarray, kernel_size: int) -> np.ndarray:
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

In [ ]:
small_data_blurr = [blur_image(img, 5) for img in small_data]
show(small_data_blurr)

# Inverse binary threshold

In [ ]:
def threshold(img: np.ndarray, limit: int) -> np.ndarray:
    return cv2.threshold(img, limit, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

In [ ]:
small_data_blurr_threshold = [threshold(img, 130) for img in small_data_blurr]
show(small_data_blurr_threshold)

# Dilation
Increase the white section created by the threshold

In [ ]:
def dilation(img, kernel_size) -> np.ndarray:
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_size, kernel_size))
    return cv2.morphologyEx(img, cv2.MORPH_ELLIPSE, kernel)

In [ ]:
small_data_completed = [dilation(img, 5) for img in small_data_blurr_threshold]
show(small_data_completed)

In [ ]:
def image_correction(img: np.ndarray, blur_size:int, limit: int, dilation_size: int) -> np.ndarray:
    blur = cv2.GaussianBlur(img, (blur_size, blur_size), 0)
    threshold = cv2.threshold(blur, limit, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (dilation_size, dilation_size))
    return cv2.morphologyEx(threshold, cv2.MORPH_ELLIPSE, kernel)

# Contour detection
The next step is to identify the plate licenses and set its boundaries